`03_output_parser`

# 출력 파서 (`OutputParser`)

- LLM의 출력을 더 유용/구조화된 형태로 변환함
- 구조화: LLM의 자유 형식 출력을 구조화된 데이터로 변환
- 일관성: 항상 일관된 출력형식 -> 후속 처리에 용이
- 유연성: 다양한 출력 형식(JSON, list, dict)로 변환 가능

## `PydanticOutputParser`

In [1]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

load_dotenv()

llm = ChatOpenAI(model='gpt-4.1-nano')

In [2]:
email_conversation = """From: 김철수 (chulsoo.kim@bikecorporation.me)
To: 이은채 (eunchae@teddyinternational.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션
"""

In [3]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """아래 이메일 내용 중 중요한 것만 추출해.
    ---
    {email_conversation}
    """
)

# print(prompt.format(email_conversation=email_conversation))
chain = prompt | llm 

print(chain.invoke({'email_conversation': email_conversation}).content)

1. 바이크코퍼레이션은 "ZENESIS" 자전거의 유통 협력 가능성에 관심이 있음.  
2. "ZENESIS" 모델의 기술 사양, 배터리 성능, 디자인 정보 요청.  
3. 미팅 제안: 1월 15일 화요일 오전 10시, 귀사 사무실에서 논의 희망.


In [4]:
class EmailSummary(BaseModel):
    person: str = Field(description='메일 보낸 사람')
    email: str = Field(description='보낸 사람의 메일주소')
    subject: str = Field(description='메일 제목')
    summary: str = Field(description='메일 본문 약')
    date: str = Field(description='메일에 언급된 미팅 날짜와 시간')

parser = PydanticOutputParser(pydantic_object=EmailSummary)
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"person": {"description": "메일 보낸 사람", "title": "Person", "type": "string"}, "email": {"description": "보낸 사람의 메일주소", "title": "Email", "type": "string"}, "subject": {"description": "메일 제목", "title": "Subject", "type": "string"}, "summary": {"description": "메일 본문 약", "title": "Summary", "type": "string"}, "date": {"description": "메일에 언급된 미팅 날짜와 시간", "title": "Date", "type": "string"}}, "required": ["person", "email", "subject", "summary", "date"]}
```


In [5]:
prompt = PromptTemplate.from_template(
    """
너는 요약의 신 어시스턴트야. 아래 질문에 맞게 답변을 한국어로 만들어줘

질문: {question}

이메일 내용: {email_conversation}

형식: {format}

"""
)

# 프롬프트 변수들 중 일부만 채우기
prompt = prompt.partial(format=parser.get_format_instructions())

In [6]:
chain = prompt | llm | parser

res = chain.invoke(
    {
        'question': '이메일 내용 중 중요한 내용을 추출해 줘 !',
        'email_conversation': email_conversation
    }
)

res

EmailSummary(person='김철수', email='chulsoo.kim@bikecorporation.me', subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안', summary="바이크코퍼레이션의 김철수 상무는 귀사의 신제품 'ZENESIS'에 관심이 있으며, 상세 브로슈어 요청과 함께 1월 15일 오전 10시에 사무실에서 협력 미팅을 제안함.", date='2024년 1월 15일 오전 10시')

In [7]:
from pprint import pprint
pprint(res.model_dump())

{'date': '2024년 1월 15일 오전 10시',
 'email': 'chulsoo.kim@bikecorporation.me',
 'person': '김철수',
 'subject': '"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안',
 'summary': "바이크코퍼레이션의 김철수 상무는 귀사의 신제품 'ZENESIS'에 관심이 있으며, 상세 브로슈어 요청과 함께 1월 "
            '15일 오전 10시에 사무실에서 협력 미팅을 제안함.'}


## CSV Parser

In [8]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()

# CSV 파서의 안내사항 확인
print(output_parser.get_format_instructions())

prompt = PromptTemplate(
    template='List 5 {subject}. \n{format_instructions}',
    input_variables=['subject'],
    partial_variables={'format_instructions': output_parser.get_format_instructions()}
)

print('----')
print(prompt.format(subject='중국집 대표메뉴'))

chain1 = prompt | llm
csv_res = chain1.invoke({'subject': '맥도날드 대표메뉴'}).content


chain2 = prompt | llm | output_parser
list_res = chain2.invoke({'subject': '국민주식'})

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`
----
List 5 중국집 대표메뉴. 
Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [9]:
print(csv_res)
print(list_res)

Big Mac, Quarter Pounder, Chicken McNuggets, McChicken, Fries
['삼성전자', 'SK하이닉스', '현대차', 'NAVER', 'LG화학']


## Structured Output Parser
- dict 형식 -> 멍청한 모델에도 적용 가능

In [10]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

In [11]:
response_schema = [
    ResponseSchema(name='answer', type='string', description='사용자의 질문에 대한 답변'),
    ResponseSchema(name='source', description='질문에 답하기 위해 사용된 "출처", "웹사이트 주소"')
]

output_parser = StructuredOutputParser.from_response_schemas(response_schema)


In [12]:
print(output_parser.get_format_instructions())

prompt = PromptTemplate(
    template='사용자 질문에 최선을 다해 답변한다.\n{format_instructions}\n{question}',
    input_variables=['questions'],
    partial_variables={'format_instructions': output_parser.get_format_instructions()}
)

print('----')
print(prompt.format(question='처서는 몇월 몇일인가?'))

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"answer": string  // 사용자의 질문에 대한 답변
	"source": string  // 질문에 답하기 위해 사용된 "출처", "웹사이트 주소"
}
```
----
사용자 질문에 최선을 다해 답변한다.
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"answer": string  // 사용자의 질문에 대한 답변
	"source": string  // 질문에 답하기 위해 사용된 "출처", "웹사이트 주소"
}
```
처서는 몇월 몇일인가?


In [13]:
chain = prompt | llm | output_parser

chain.invoke({'question': '처서는 언제야?'})

{'answer': '처서는 일반적으로 1월 1일을 의미하며, 새해의 시작을 기념하는 날입니다.',
 'source': 'https://ko.wikipedia.org/wiki/%EC%B2%98%EC%84%9C'}

In [14]:
for t in (prompt | llm).stream({'question': '처서는 언제야?'}):
    print(t.content, end='', flush=True)

```json
{
	"answer": "처서는 일반적으로 매년 10월 15일부터 11월 15일까지입니다. 다만, 특정 지역이나 문화에 따라 차이가 있을 수 있으니 참고하시기 바랍니다.",
	"source": "https://ko.wikipedia.org/wiki/%ED%83%80%EC%8A%A4%ED%8A%B8%EC%86%8C"
}
```

## DataFrame Output Parser
- 원하는 Dataframe을 분석할 수 있음.
- 답변 또한 판다스 DF로 반환

In [15]:
from langchain.output_parsers import PandasDataFrameOutputParser
import seaborn as sns

titanic_df = sns.load_dataset('titanic')

In [16]:
parser = PandasDataFrameOutputParser(dataframe=titanic_df)

print(parser.get_format_instructions())

The output should be formatted as a string as the operation, followed by a colon, followed by the column or row to be queried on, followed by optional array parameters.
1. The column names are limited to the possible columns below.
2. Arrays must either be a comma-separated list of numbers formatted as [1,3,5], or it must be in range of numbers formatted as [0..4].
3. Remember that arrays are optional and not necessarily required.
4. If the column is not in the possible columns or the operation is not a valid Pandas DataFrame operation, return why it is invalid as a sentence starting with either "Invalid column" or "Invalid operation".

As an example, for the formats:
1. String "column:num_legs" is a well-formatted instance which gets the column num_legs, where num_legs is a possible column.
2. String "row:1" is a well-formatted instance which gets row 1.
3. String "column:num_legs[1,2]" is a well-formatted instance which gets the column num_legs for rows 1 and 2, where num_legs is a p

In [17]:
prompt = PromptTemplate(
    template='Answer the user query.\n{format_instructions}\n{query}',
    input_variables=['query'],
    partial_variables={'format_instructions': parser.get_format_instructions()},
)

chain = prompt | llm | parser

In [18]:
res = chain.invoke({'query': 'age 컬럼을 조회해 줘'})

In [19]:
res = chain.invoke({'query': 'age 컬럼의 평균을 구해줘'})
res['mean']

np.float64(29.69911764705882)

In [ ]:
res = chain.invoke({'query': 'fare 컬럼 평균을 구해줘'})
res['mean']

## Datetime Output Parser
- 날짜 출력용

In [22]:
from langchain.output_parsers import DatetimeOutputParser

parser = DatetimeOutputParser(format='%Y-%m-%d')

prompt = PromptTemplate(
    template='사용자 질문에 답해라.\n{format_instructions}\n{question}',
    input_variables=['question'],
    partial_variables={
        'format_instructions': output_parser.get_format_instructions()
    }
)

In [30]:
chain = prompt | llm | parser
# datetime 인스턴스로 만들어 줌 !
chain.invoke({'question': '내가 집을 사게 될 날은?'})

datetime.datetime(2024, 12, 15, 0, 0)

## Enum Output Parser (Enumerated Type -> 열거형)
- 정해진 목록 중에 답을 고르도록

In [37]:
from langchain.output_parsers import EnumOutputParser
from enum import Enum

llm = ChatOpenAI(model='gpt-4.1-nano')
class Colors(Enum):
    RED = '빨간색'
    GREEN = '초록색'
    BLUE = '파란색'
    # YELLOW = '노란색'


parser = EnumOutputParser(enum=Colors)

prompt = PromptTemplate(
    template='다음 물체는 어떤 색깔에 가깝나요 ?\n{object}\n\nInstructions:{instructions}',
    input_variables=['object'],
    partial_variables={'instructions': parser.get_format_instructions()}
)

chain = prompt | llm | parser

chain.invoke({'object': '병아리'})

<Colors.GREEN: '초록색'>

## 감정분석 Agent

In [10]:
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from pydantic import BaseModel, Field


# 1. pydantic 클래스(답변 형태)
class Classification(BaseModel):
    sentiment: str = Field(description='글의 감정')
    aggressiveness: int = Field(description='얼마나 공격적인지 1 ~ 10점으로 판단')
    language: str = Field(description='작성된 글의 언어')
    remarks: str = Field(description='명사형으로 내용 요약(20글자 이내) 간략하게')


# 2. LLM
llm = ChatOpenAI(model='gpt-4.1-nano', temperature=0)
structured_llm = llm.with_structured_output(Classification)

# 3. 프롬프트 정의
prompt = ChatPromptTemplate([
    ('system', '너는 텍스트에서 감정, 공격성, 언어, 특이사항을 추출하는 분류기야. 지금까지의 대화내용을 종합해서 한국어로 대답해야해'),
    MessagesPlaceholder(variable_name='chat_history'),
    ('human', '{input}'),
    MessagesPlaceholder(variable_name='agent_scratchpad')
])

# 4. 메모리 정의
memory = ConversationBufferMemory(
    return_messages=True,
    memory_key='chat_history'
)

# 5. Agent 조립
agent = create_openai_tools_agent(
    llm=llm,
    tools=[],
    prompt=prompt
)

agent_executor = AgentExecutor(
    agent=agent,
    tools=[],
    memory=memory,
    verbose=True
)

from langchain_core.runnables import RunnableLambda

inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"

pipeline = (
    agent_executor
    | RunnableLambda(lambda d: d['output'])
    | structured_llm
)

pipeline.invoke({'input': inp})



> Entering new AgentExecutor chain...
¡Qué alegría escuchar eso! Parece que estás muy feliz y emocionado por la nueva amistad.

> Finished chain.


Classification(sentiment='feliz', aggressiveness=2, language='es', remarks='Respuesta positiva expresando alegría por la buena noticia.')

In [11]:
agent_executor.invoke({'input': '난 요즘 고민이 있어'})



> Entering new AgentExecutor chain...
네가 요즘 고민이 있다는 말이구나. 어떤 고민인지 말해줄 수 있니? 도움이 될 수 있도록 들어줄게.

> Finished chain.


{'input': '난 요즘 고민이 있어',
 'chat_history': [HumanMessage(content='Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!', additional_kwargs={}, response_metadata={}),
  AIMessage(content='¡Qué alegría escuchar eso! Parece que estás muy feliz y emocionado por la nueva amistad.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='난 요즘 고민이 있어', additional_kwargs={}, response_metadata={}),
  AIMessage(content='네가 요즘 고민이 있다는 말이구나. 어떤 고민인지 말해줄 수 있니? 도움이 될 수 있도록 들어줄게.', additional_kwargs={}, response_metadata={})],
 'output': '네가 요즘 고민이 있다는 말이구나. 어떤 고민인지 말해줄 수 있니? 도움이 될 수 있도록 들어줄게.'}